# Movie Recomender System

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

: 

In [ ]:
movies = pd.read_csv('./Dataset/tmdb_5000_movies.csv')
credits = pd.read_csv('./Dataset/tmdb_5000_credits.csv')

In [ ]:
movies = movies.merge(credits, on='title')

In [ ]:
movies.columns

### The column we will be work on
- generes
- id
- keywords
- title
- overview
- cast
- crew 

In [ ]:
movies = movies[['movie_id','title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

In [ ]:
movies.isnull().sum()

In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies.duplicated().sum()

In [ ]:
movies['genres'][0]

In [ ]:
#'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'
#['Action', 'Adventure', 'Fantasy', 'Science Fiction']

- Basically  '[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]' This is a string list which is not iterable so we have to convert it into a list 


In [ ]:
# This package will help to convert string representation of list into list
import ast

In [ ]:
# This is the function which will convert the string list into normal list
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return  L 

In [ ]:
movies['genres'] = movies['genres'].apply(convert)

In [ ]:
movies['keywords'] = movies['keywords'].apply(convert)

In [ ]:
movies['cast'][0]

In [ ]:
def convert3(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter!=3:
            L.append(i['name'])
            counter+=1
        else:
            break
    return  L 

In [ ]:
movies['cast'] = movies['cast'].apply(convert3)

In [ ]:
def convertCrew(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            L.append(i['name'])
            break
    return L

In [ ]:
movies.shape

In [ ]:
movies['crew'] = movies['crew'].apply(convertCrew)

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [ ]:
movies.columns

In [ ]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [ ]:
new_df = movies[['movie_id','title','tags']]    

In [ ]:
new_df['tags'] =  new_df['tags'].apply(lambda x: " ".join(x))

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())

In [ ]:
import nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
new_df['tags'] = new_df['tags'].apply(stem)

In [ ]:
new_df['tags'][0]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')


In [ ]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
cosine_sim = cosine_similarity(vectors)

In [ ]:
new_df[new_df['title'] == 'Avatar'].index[0]

In [ ]:
def recomend(movie):
    movie_index = new_df[new_df['title']==movie].index[0]
    distances = cosine_sim[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:10]
    
    for i in movies_list:
        print(i)

In [ ]:
recomend('Avatar')

In [ ]:
import pickle

In [ ]:
pickle.dump(new_df,open('movies.pkl','wb'))

In [ ]:
pickle.dump(cosine_sim,open('similarity.pkl','wb'))